In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [2]:
import os
import json
from dotenv import dotenv_values

import pandas as pd
from tqdm import tqdm

In [3]:
config = dotenv_values("./.env")
config.items()

odict_items([('SCRAPED_DATA_DIR', '../../Crawler/output'), ('EMBED_DATA_DIR', './output'), ('llm_model', 'google-bert/bert-base-uncased')])

In [4]:
pageNodes = []

dataFiles = os.listdir(config["SCRAPED_DATA_DIR"])
for file in (dataFiles):
    with open(os.path.join(config["SCRAPED_DATA_DIR"], file)) as f:
        data = json.load(f)
        # print(data)
        pageNodes.append({
            "doc": " ".join(data['doc']),
            "headings":" ".join(data['headings']),
            "url": data['url'],
            'previewTitle': data['previewTitle'],
            "preview":data["preview"],
        })
# print(pageNodes)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(config['llm_model'])
model = AutoModel.from_pretrained(config['llm_model'])

/home/sunaam/miniconda3/envs/nlp/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
input_text = "Hello, My name is Sunaam"
input2 = "Hello"
input3 = "name is sunaam, hello"
input4 = 'sunaam'
input5 = 'name'
input6 = 'name is taha'

In [7]:
def compute_embeddings(input_text, tokenizer, model):
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    embeddings = outputs.last_hidden_state
    mean_embeds = torch.mean(embeddings, dim=1)
    cls_embeds = embeddings[:,0,:]
    return {
        "mean": mean_embeds,
        "cls":cls_embeds
    }

In [8]:
def compute_similarity(embed1, embed2):
    mean_cosine = F.cosine_similarity(embed1['mean'], embed2['mean'], dim=1)
    cls_cosine = F.cosine_similarity(embed1['cls'], embed2['cls'], dim=1)
    return {
        "mean": mean_cosine.item(),
        "cls":cls_cosine.item()
    }

In [11]:
if not os.path.exists(config['EMBED_DATA_DIR']):
    os.makedirs(config['EMBED_DATA_DIR'])
    
count = 0

for page in pageNodes:
    data_to_write = {
        "embed1": compute_embeddings(page['preview'], tokenizer, model)['mean'].squeeze(0).numpy().tolist(),
        "embed2":compute_embeddings(page['previewTitle'], tokenizer, model)['mean'].squeeze(0).numpy().tolist(),
        "previewTitle": page['previewTitle'],
        'preview':page['preview'],
        "url":page['url']
    }
    with open(f"{config['EMBED_DATA_DIR']}/{count}_page.json", 'w') as file:
        json.dump(data_to_write, file, indent=4)
    count += 1